# Star_Galaxy_HDBSCAN_v1
**Author:** Kalea Sebesta<br>
**Date:** 10 October 2018<br>

**Purpose:** The purpose to this notebook is to explore the application of HDBSCAN on the Sloan Digital Sky dataset from Kaggle. https://www.kaggle.com/lucidlenn/sloan-digital-sky-survey

**Description of Algorithm:** HDBSCAN stands for Hierarchical Density-Based Spatial Clustering of Applications with Noise. This approach uses a technique that extracts flat clusterings that are considered stable clusters. According to the HDBSCAN documentation at https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html "We can break it out into a series of steps:
1) Transform the space according to the density/sparsity.
2) Build the minimum spanning tree of the distance weighted graph.
3) Construct a cluster hierarchy of connected components.
4) Condense the cluster hierarchy based on minimum cluster size.
5) Extract the stable clusters from the condensed tree."

Lets explain these steps in greater detail for a better understanding. Transforming the space essentially is the way to "find the islands of higher density amongst the sea of sparser noise". This makes the algorthim robust to noise which is extremely useful and important. In order to accomplish this there are two distance metrics that are being used, **core distance** and **mutual reachablity**. Core distance is the distance from the center of the cluster to the boundary of the cluster. This can changed based on how many k neighbors are in the cluster. Essentially, the core distance is the distances from a point (core) to the k nearest neighbor. Mutual reachability is the longest distance between two point looking at the three following options: 
- the core distance of the first point (core(a) 
- the core distance of the second point (core(b))
- the distance between point a and point b

The mutual reachability gives the information needed to build the minimum spanning tree. Along with the mutual reachability values the practioner needs to choose a threshold value. Create a weighted graph using the mutual reachability and connect all points using the densities from the mutual reachability as edges. If an edge is above the chosen threshold then drop the edge, in this manner the spanning tree is created. It is important to note that the threshold value should start high and then adjust it lower to find the optimal value.

Next, building the cluster heirarchy uses the information from the minimum spanning tree. The cluster heirarchy is essentially creating connected components from the minimum spanning tree. This can be done by sorting the edges of the tree by increasing order of distance and then iterating through to create/merge clusters together for each edge.

From there the HDBSCAN algorithm condenses the cluster tree. This is accomplished by using the prameter **minimum cluster size** as the "fall out point". This means that at each split in the hierarchy if the new cluster that was created by the split has few points than the minimum cluster size than it is considered noise. If both clusters from the split have the minimum cluster size or larger number of points than they are both considered true clusters.

Lastly, clusters are extracted from the cluster tree. Ideally choosing clusters that have a longer lifetime.

It is important to note that HDBSCAN has 15 parameter to tune which makes this algorithm not only robust to noise but also applicable to many differing data.

For the purpose of this script, minimum cluster size and metric will be the two parameters that I will initially investigate and tune. The defualt parameters of HDBSCAN are as follows:

### Data Desciption
According to the Sloan Digital Sky Survey/ SkyServer Glossary the following features of the dataset are described as follows:
- objid: The long object identification, which is a bit-encoded integer of run,rerun, camcol, field, object. When the data is reprocessed (rerun), this number will change! IMPORTANT NOTE: For spectroscopic objects, there are two possible choices for the matching photometric measurement: TargetObjID is the photometric object identification number of the corresponding photometric object when targeting was run, and BestObjID, which points to the best imaging and processing of the photometry.
- ra: The SDSS has two sets of coordinates it uses which are specially designed for the survey geometry. spherical coordinates (corresponding to **RA** and Dec)
- dec: The SDSS has two sets of coordinates it uses which are specially designed for the survey geometry. spherical coordinates (corresponding to RA and **Dec**)
- u: The SDSS uses five filters: u,g,r,i,z. Filter u is Ultraviolet with a wavelength of 3543 Angstromes
- g: The SDSS uses five filters: u,g,r,i,z. Green (g) 4770 Angstromes
- r: The SDSS uses five filters: u,g,r,i,z. Red (r) 6231 Angstromes
- i: The SDSS uses five filters: u,g,r,i,z. Near Infrared (i) 7625 Angstromes
- z: The SDSS uses five filters: u,g,r,i,z. Infrared (z) 9134Angstromes
- run: A Run is just a length of a strip observed in a single contiguous observing pass scan, bounded by lines of mu and nu. A strip covers a great circle region from pole to pole; this cannot be observed in one pass. The fraction of a strip observed at one time (limited by observing conditions) is a Run. Runs can (and usually do) overlap at the ends. Like strips, it takes a pair of runs to fill in a length of a stripe. This is why you may read about data taken from "Runs 752/756" or some similar terminology. However, each individual run does contain 6 camcols spanning the same range of nu, but not delimited by eta. These run pairs might not have the same starting and ending nu coordinates.
- rerun: A reprocessing of an imaging run. The underlying imaging data is the same, just the software version and calibration may have changed.
- camcol: A Camcol is the output of one camera column as part of a Run. Therefore, 1 Camcol = 1/6 of a Run. It is also a portion of a scanline.
- field: A field is a part of a camcol that is processed by the Photo pipeline at one time. Fields are 2048x1489 pixels; a field consists of the frames in the 5 filters for the same part of the sky. Fields overlap each other by 128 rows; primaries are decided when Chunks are resolved (using objects between rows 64 and1425 as primaries). A field at the edge of a Chunk may in fact be included in 2 (or more) Chunks.
- specobjid: A unique bit-encoded 64-bit ID used for spectroscopic objects. It is generated from plateid, mjd, and fiberid. Completely independent of any photometric enumeration system.
- class: Outcome variable (STAR, GALAXY, QSO)
- redshift:
- plate: SDSS has the largest multi-fiber spectrograph in operation in the world, observing 640 objects simultaneously. This is done by drilling holes in round aluminum plates at the positions of objects of interest, and plugging optical fibers into each hole.
- mjd: Part of the specobjid
- fiberid: The SDSS spectrograph uses optical fibers to direct the light from individual objects to the slithead. Each object is assigned a corresponding fiberID. The fibers are 3 arcsec in diameter in the source plane. Each fiber is surrounded by a large sheath which prevents any pair of fibers from being placed closer than 55 arcsec on the same plate.

### Import Packages

In [205]:
import pandas as pd
import hdbscan 
import numpy as np
import pandas_profiling as pp
from sklearn import preprocessing
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from functools import reduce

### Read in File & Exploratory Analysis
After the file is read into a pandas dataframe, I check for missing values. In this particular dataset there are no missing values. From there I look at the descriptive statistics for the numerical variables. After looking at the descriptive statistics I utlize the the pandas profiling package to run a report on the dataframe. This gives the histograms for the distribution of the variables, spearman and pearson corralations, and descriptive stats. From this it seen that objid and rerun are constants and should be dropped from any analysis that is performed. Also, specobjid is a unique identifier and a composition variable with plateid, mjd, and fiberid and thus is should also be dropped prior to analysis. i is highly correlated with r (ρ = 0.97767) therefore should be rejected, mjd is highly correlated with plate (ρ = 0.96688) thus it too should be rejected as well as, plate since it is highly correlated with specobjid (ρ = 1), r since it is highly correlated with g (ρ = 0.95811), and z since it is highly correlated with i (ρ = 0.98151).

In [219]:
#read in file
df = pd.read_csv('/Users/kaleasebesta/Downloads/Skyserver_SQL2_27_2018 6_51_39 PM.csv')

In [221]:
df.isnull().sum()

objid        0
ra           0
dec          0
u            0
g            0
r            0
i            0
z            0
run          0
rerun        0
camcol       0
field        0
specobjid    0
class        0
redshift     0
plate        0
mjd          0
fiberid      0
dtype: int64

In [222]:
df.describe()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.237650e+18,175.529987,14.836148,18.619355,17.371931,16.840963,16.583579,16.422833,981.034800,301.0,3.648700,302.380100,1.645022e+18,0.143726,1460.986400,52943.533300,353.069400
std,1.577039e+05,47.783439,25.212207,0.828656,0.945457,1.067764,1.141805,1.203188,273.305024,0.0,1.666183,162.577763,2.013998e+18,0.388774,1788.778371,1511.150651,206.298149
min,1.237650e+18,8.235100,-5.382632,12.988970,12.799550,12.431600,11.947210,11.610410,308.000000,301.0,1.000000,11.000000,2.995780e+17,-0.004136,266.000000,51578.000000,1.000000
25%,1.237650e+18,157.370946,-0.539035,18.178035,16.815100,16.173333,15.853705,15.618285,752.000000,301.0,2.000000,184.000000,3.389248e+17,0.000081,301.000000,51900.000000,186.750000
50%,1.237650e+18,180.394514,0.404166,18.853095,17.495135,16.858770,16.554985,16.389945,756.000000,301.0,4.000000,299.000000,4.966580e+17,0.042591,441.000000,51997.000000,351.000000
75%,1.237650e+18,201.547279,35.649397,19.259232,18.010145,17.512675,17.258550,17.141447,1331.000000,301.0,5.000000,414.000000,2.881300e+18,0.092579,2559.000000,54468.000000,510.000000
max,1.237650e+18,260.884382,68.542265,19.599900,19.918970,24.802040,28.179630,22.833060,1412.000000,301.0,6.000000,768.000000,9.468830e+18,5.353854,8410.000000,57481.000000,1000.000000


In [16]:
pp.ProfileReport(df)

Number of variables,18
Number of observations,10000
Total Missing (%),0.0%
Total size in memory,1.4 MiB
Average record size in memory,144.0 B
Numeric,10
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,7


### Cleaning 
To clean this dataset I will drop the constant variables as well as the variables that were found to be highly correlated during the exploratory phase of analysis. Also, the 'class' variable is dropped in order to separate out the features from the target variable (class).

In [224]:
#cleaning
features = df.drop(['class', 'i', 'z', 'mjd', 'objid', 'plate', 'r', 'rerun', 'specobjid'], axis = 1)

## Create function to test the different metric parameters of HDBSCAN
This function is created to look at different metrics to run the clustering algorithm. It then takes the cluster label that is found using the HDBSCAN algorithm this information is stored in a new column labeled 'label'. It calculates the number of unique clusters for the three different classes (star, qso, and galaxy). It also calculates how many labels are shared in all three classes. From there the percentage of noise in each class is found and displayed as is the amount of overlap in cluster labels for the classes.

In [225]:
def hdbscan_clusters(metric_type):
    #create clusterer algorithm for the specific metric and fit it to the features of the data
    clusterer = hdbscan.HDBSCAN(metric = metric_type)
    clusterer.fit(features)
    
    #create new dataframe that will contain the original dataframe and a new column 'label' 
    #which holds the label found from the clustering alg.
    df_new = df
    df_new['label'] = clusterer.labels_
    
    #display information to the user
    print('Metric: {}'.format(metric_type))
    print('Unique Labels in Star Group: {}'.format(len(df_new[(df_new['class']=='STAR')].label.unique())))
    print('Unique Labels in QSO Group: {}'.format(len(df_new[(df_new['class']=='QSO')].label.unique())))
    print('Unique Labels in Galaxy Group: {}'.format(len(df_new[(df_new['class']=='GALAXY')].label.unique())))
    
    #identify the cluster labels for each class
    star = df_new[(df_new['class']=='STAR')].label.unique()
    qso = df_new[(df_new['class']=='QSO')].label.unique()
    galaxy = df_new[(df_new['class']=='GALAXY')].label.unique()
    
    #display information to the user
    print('Unique Labels that are in Star, QSO, and Galaxy Group: {}'.format(len(reduce(np.intersect1d, ([star], [galaxy], [qso])))))
    print('Percentage of noise label in Star Group: {}'.format(len(df_new[(df_new['class']=='STAR') & (df_new['label'] == -1)])/float(len(df_new))))
    print('Percentage of noise label in QSO Group: {}'.format(len(df_new[(df_new['class']=='QSO') & (df_new['label'] == -1)])/float(len(df_new))))
    print('Percentage of noise label in Galaxy Group: {}'.format(len(df_new[(df_new['class']=='GALAXY') & (df_new['label'] == -1)])/float(len(df_new))))
    print('Percentage of cluster overlap in Star Group: {}'.format(len(df_new.loc[(df_new['label'].isin(reduce(np.intersect1d, ([star], [galaxy], [qso])))) & (df_new['class'] == 'STAR')])/float(len(df_new['class'] == 'STAR'))))
    print('Percentage of cluster overlap in QSO Group: {}'.format(len(df_new.loc[(df_new['label'].isin(reduce(np.intersect1d, ([star], [galaxy], [qso])))) & (df_new['class'] == 'QSO')])/float(len(df_new['class'] == 'QSO'))))
    print('Percentage of cluster overlap in Galaxy Group: {}'.format(len(df_new.loc[(df_new['label'].isin(reduce(np.intersect1d, ([star], [galaxy], [qso])))) & (df_new['class'] == 'GALAXY')])/float(len(df_new['class'] == 'GALAXY')))) 

### Apply Function 
Apply the hdbscan cluster function that was created to loop through all the metric possibilities within the hdbscan algorithm parameter to compare the results based on the metric changing.

In [226]:
#create list of metrics to loop through
metric_list=['braycurtis','canberra','chebyshev','cityblock','dice','euclidean','hamming',
             'infinity','jaccard','kulsinski','l1','l2','manhattan',
             'matching','p','rogerstanimoto','russellrao',
             'sokalmichener','sokalsneath']

In [276]:
#loop through the metric list and apply the clustering function that was created to compare results
for metric in metric_list:
    hdbscan_clusters(metric)

Metric: braycurtis
Unique Labels in Star Group: 516
Unique Labels in QSO Group: 294
Unique Labels in Galaxy Group: 561
Unique Labels that are in Star, QSO, and Galaxy Group: 227
Percentage of noise label in Star Group: 0.0937
Percentage of noise label in QSO Group: 0.0358
Percentage of noise label in Galaxy Group: 0.163
Percentage of cluster overlap in Star Group: 0.2255
Percentage of cluster overlap in QSO Group: 0.0749
Percentage of cluster overlap in Galaxy Group: 0.3165
Metric: canberra
Unique Labels in Star Group: 41
Unique Labels in QSO Group: 38
Unique Labels in Galaxy Group: 102
Unique Labels that are in Star, QSO, and Galaxy Group: 2
Percentage of noise label in Star Group: 0.0584
Percentage of noise label in QSO Group: 0.0401
Percentage of noise label in Galaxy Group: 0.1341
Percentage of cluster overlap in Star Group: 0.137
Percentage of cluster overlap in QSO Group: 0.0402
Percentage of cluster overlap in Galaxy Group: 0.1342
Metric: chebyshev
Unique Labels in Star Group: 4

Metric: sokalsneath
Unique Labels in Star Group: 1
Unique Labels in QSO Group: 1
Unique Labels in Galaxy Group: 2
Unique Labels that are in Star, QSO, and Galaxy Group: 1
Percentage of noise label in Star Group: 0.0
Percentage of noise label in QSO Group: 0.0
Percentage of noise label in Galaxy Group: 0.0
Percentage of cluster overlap in Star Group: 0.4152
Percentage of cluster overlap in QSO Group: 0.085
Percentage of cluster overlap in Galaxy Group: 0.4979


### Interpret Results
There were a few metrics that produces no noise and very few clusters for the various classes. To investigate these clusters a new function was created that would display the clusters labels. A metric subset was identified for those metrics that only resulted in the star, qso, and galaxy only having 1 or 2 labels. When these metrics were used to loop through the new function it was seen that all star and qso were being labeled as 0 and galaxy was being labeled as 0 or 1. Therefore, using any one of the metrics from the subset metric list if a datapoint is labeled as 1 it is certain that is is a galaxy. These metrics would not be useful if a star or qso wanted to be identified with high accuracy.

In [218]:
#create function to return the cluster labels for the hdbscan algorithm
def cluster_labels_output(metric):
    #create clusterer algorithm for the specific metric and fit it to the features of the data
    clusterer = hdbscan.HDBSCAN(metric = metric)
    clusterer.fit(features)
    
    #create new dataframe that will contain the original dataframe and a new column 'label' 
    #which holds the label found from the clustering alg.    
    df_new = df
    df_new['label'] = clusterer.labels_
    
    #display information
    print('Metric type: {}'.format(metric))
    print('Array of cluster arrays for Star:{}'.format(df_new[(df_new['class']=='STAR')].label.unique()))
    print('Array of cluster arrays for QSO:{}'.format(df_new[(df_new['class']=='QSO')].label.unique()))
    print('Array of cluster arrays for Galaxy:{}'.format(df_new[(df_new['class']=='GALAXY')].label.unique()))
    

In [235]:
#loop through those metric that only had 1 or 2 clusters
metric_subset = ('dice', 'jaccard', 'kulsinski', 'matching', 'rogerstanimoto', 
                'sokalmichener', 'sokalsneath')

In [236]:
#loop through the metric subset list and apply the clustering function that was created to compare results
for metric in metric_subset:
    cluster_labels_output(metric)

Metric type: dice
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: jaccard
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: kulsinski
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: matching
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: rogerstanimoto
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: sokalmichener
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]
Metric type: sokalsneath
Array of cluster arrays for Star:[0]
Array of cluster arrays for QSO:[0]
Array of cluster arrays for Galaxy:[0 1]


### Create Function to Find Labels that are Unique across the Class
Since there is overlap in the cluster labels I created a function that would find the cluster labels that only exist in each class. This will allow for better insight when new data is read in without a class label. Essentially the cluster label could be mapped back to determine the appropriate class label.

In [274]:
#create function to see which cluster labels are ONLY in each class given the specific metric
def clusters_specific_class(metric_type):
    #create clusterer algorithm for the specific metric and fit it to the features of the data
    clusterer = hdbscan.HDBSCAN(metric = metric_type)
    clusterer.fit(features)
    
    #create new dataframe that will contain the original dataframe and a new column 'label' 
    #which holds the label found from the clustering alg.
    df_new = df
    df_new['label'] = clusterer.labels_
    print('Metric: {}'.format(metric_type))
    
    #find which labels are in the different classes and those shared by all three
    star = df_new[(df_new['class']=='STAR')].label.unique()
    qso = df_new[(df_new['class']=='QSO')].label.unique()
    galaxy = df_new[(df_new['class']=='GALAXY')].label.unique()
    
    un_star = np.setdiff1d(star,qso)   
    un_star = np.setdiff1d(un_star, galaxy)
    
    un_qso = np.setdiff1d(qso, star)   
    un_qso = np.setdiff1d(un_qso, galaxy)
    
    un_gal = np.setdiff1d(galaxy,star)   
    un_gal = np.setdiff1d(un_gal, qso)
    
    #display the labels that are unique only to each class, thus not existing in the other two classes
    print('Unique cluster labels in Star: {}'.format(un_star))
    print('Unique cluster labels in QSO: {}'.format(un_qso))
    print('Unique cluster labels in Galaxy: {}'.format(un_gal))
    

### Interpret Results
Applying the function that I created, the metric 'canberra' is the only metric that identifies unique cluster labels to each class type. From initial inspection canberra metric seems to be a good starting place. Next steps should include tuning the minimum_cluster_size to reduce the nosie within the cluster labels as well as overlap in the labels.<br>

Metric: canberra
- 37 cluster labels unique to ONLY Star
- 20 cluster labels unique to ONLY QSO
- 82 cluster labels unique to ONLY galaxy
- Unique Labels in Star Group: 41
- Unique Labels in QSO Group: 38
- Unique Labels in Galaxy Group: 102
- Unique Labels that are in Star, QSO, and Galaxy Group: 2
- Percentage of noise label in Star Group: 0.0584
- Percentage of noise label in QSO Group: 0.0401
- Percentage of noise label in Galaxy Group: 0.1341
- Percentage of cluster overlap in Star Group: 0.137
- Percentage of cluster overlap in QSO Group: 0.0402
- Percentage of cluster overlap in Galaxy Group: 0.1342
- Unique cluster labels in Star: [ 0  1  3  8  9 12 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 30 32 33
 41 42 46 47 48 49 68 69 70 85 86 91 92]
- Unique cluster labels in QSO: [  4  29  31  34  44  57  58  63  71  72  73  77  79  80  88  93  94  95
  96 100]
- Unique cluster labels in Galaxy: [  5   6  10  36  38  40  43  45  50  51  52  53  54  55  56  59  60  62
  65  66  67  74  75  76  81  82  83  84  89  97  98  99 103 104 105 106
 107 108 110 111 112 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 146
 147 148 149 150 151 152 153 154 156 157]

In [275]:
#apply function to all metrics to identify the cluster labels that are unique to each class.
for metric in metric_list:
    clusters_specific_class(metric)

Metric: braycurtis
Unique cluster labels in Star: [  1   4   5  11  16  17  24  25  26  58  67  68 102 103 188 207 222 224
 258 266 268 269 275 282 345 355 371 382 427 429 435 437 448 458 469 474
 488 489 501 503 528 562 571]
Unique cluster labels in QSO: []
Unique cluster labels in Galaxy: [ 30  44  70  76  77 112 126 130 131 132 133 160 168 180 183 196 198 210
 219 227 236 240 261 290 314 338 342 347 348 351 389 401 415 434 451 467
 495 510 521 527 529 533 543 560 563 574 582 588 590]
Metric: canberra
Unique cluster labels in Star: [ 0  1  3  8  9 12 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 30 32 33
 41 42 46 47 48 49 68 69 70 85 86 91 92]
Unique cluster labels in QSO: [  4  29  31  34  44  57  58  63  71  72  73  77  79  80  88  93  94  95
  96 100]
Unique cluster labels in Galaxy: [  5   6  10  36  38  40  43  45  50  51  52  53  54  55  56  59  60  62
  65  66  67  74  75  76  81  82  83  84  89  97  98  99 103 104 105 106
 107 108 110 111 112 114 115 116 117 118 119 120 121 1